<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/%EA%B8%B0%EC%83%81%ED%98%84%EC%83%81%EC%97%90%EB%94%B0%EB%A5%B8%ED%83%9C%EC%96%91%EA%B4%91%EB%B0%9C%EC%A0%84%EB%9F%89%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall --y kaggle
!pip install --upgrade pip
!pip  install kaggle==1.5.6

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle -v

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 13.2MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 4.9 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=c99001f69939600a3d7d28161910daf3765cf3e589618d29447bf4be71d8d926
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Kaggle API 1.5.6


In [2]:
!kaggle competitions download -c solarenergy-meteorologicalphenomenon2
!unzip solarenergy-meteorologicalphenomenon2.zip

  0% 0.00/6.51k [00:00<?, ?B/s]
100% 6.51k/6.51k [00:00<00:00, 6.56MB/s]
Archive:  solarenergy-meteorologicalphenomenon2.zip
  inflating: Solar_SubmitForm_2.csv  
  inflating: Solar_TestData_2.csv    
  inflating: Solar_TrainData_3.csv   


In [43]:
import pandas as pd
import numpy as np
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
import random

from sklearn import preprocessing

In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [45]:
learning_rate = 1e-3
training_epochs = 900
batch_size = 50
Scaler = preprocessing.StandardScaler()

In [46]:
train = pd.read_csv('Solar_TrainData_3.csv',header=None,skiprows=1,usecols=range(0,9))
train = train.dropna()
print(train.head(10))
print(train.info())

            0    1    2    3      4    5     6    7      8
0  2018-01-01  0.9  0.0  1.5  0.479  8.4  7.13  0.0  1.229
1  2018-01-02  0.7  0.0  0.7  0.569  8.1  6.70  0.0  1.034
2  2018-01-03  2.0  0.0  1.5  0.265  8.6  7.35  0.0  1.193
3  2018-01-04  1.4  0.0  1.0  0.321  4.0  4.79  0.0  0.859
4  2018-01-05  2.4  0.0  1.3  0.379  7.1  6.54  0.0  1.197
5  2018-01-06 -0.3  0.0  0.8  0.523  8.5  7.64  0.0  1.268
6  2018-01-07 -0.3  0.9  0.5  0.619  1.0  2.81  0.0  0.419
7  2018-01-08  3.2  1.5  1.1  0.814  0.0  1.23  0.0  0.231
8  2018-01-09  1.8  0.0  2.6  0.450  7.7  6.81  0.0  0.685
9  2018-01-10 -1.7  0.5  2.3  0.573  4.3  4.31  0.0  0.640
<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 349
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       350 non-null    object 
 1   1       350 non-null    float64
 2   2       350 non-null    float64
 3   3       350 non-null    float64
 4   4       350 non-null

In [47]:
test = pd.read_csv('Solar_TestData_2.csv',header=None,skiprows=1,usecols=range(0,8))

print(test.head(10))
print(test.info())

            0     1     2    3      4     5      6    7
0  2018-01-11  -5.4   0.0  1.5  0.465   8.7   7.78  0.0
1  2018-02-16   3.4   0.0  0.8  0.309   5.2   6.63  0.0
2  2018-03-04   9.4  13.1  0.8  0.859   0.0   2.40  0.0
3  2018-03-28  13.4   0.0  1.2  0.611  10.7  13.74  0.0
4  2018-04-27  16.1   0.0  1.1  0.583  11.1  24.00  0.0
5  2018-05-11  16.5   0.0  1.2  0.603  10.3  24.66  0.0
6  2018-05-30  22.1   0.0  1.2  0.765   6.1  18.86  0.0
7  2018-06-12  18.5  19.4  0.9  0.830   1.3   9.28  0.0
8  2018-06-25  24.6   0.0  1.2  0.650  10.2  22.89  0.0
9  2018-07-28  28.6   0.0  1.1  0.779   3.8  13.54  0.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       16 non-null     object 
 1   1       16 non-null     float64
 2   2       16 non-null     float64
 3   3       16 non-null     float64
 4   4       16 non-null     float64
 5   5       16 non-nul

In [48]:
x_train = train.iloc[:,1:7]
y_train = train.iloc[:,[-1]]

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
print(x_train.shape)
print(y_train.shape)

torch.Size([350, 6])
torch.Size([350, 1])


In [49]:
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)

data_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)


In [50]:
class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x)) 
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)

In [51]:
linear1 = torch.nn.Linear(6,32, bias = True) # feature
linear2 = torch.nn.Linear(32,32, bias = True)
linear3 = torch.nn.Linear(32,32, bias = True)
linear4 = torch.nn.Linear(32,16, bias = True)
linear5 = torch.nn.Linear(16,16, bias = True)
linear6 = torch.nn.Linear(16,16, bias = True)
linear7 = torch.nn.Linear(16,8, bias = True)
linear8 = torch.nn.Linear(8,8, bias = True)
linear9 = torch.nn.Linear(8,8, bias = True)
linear10 = torch.nn.Linear(8,1, bias = True)

mish = Mish()

In [52]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)
torch.nn.init.xavier_uniform_(linear6.weight)
torch.nn.init.xavier_uniform_(linear7.weight)
torch.nn.init.xavier_uniform_(linear8.weight)
torch.nn.init.xavier_uniform_(linear9.weight)
torch.nn.init.xavier_uniform_(linear10.weight)

Parameter containing:
tensor([[-0.0369, -0.1342,  0.7287,  0.6699, -0.5252, -0.4322, -0.6342,  0.2139]],
       requires_grad=True)

In [53]:
model = torch.nn.Sequential(linear1,mish,
                            linear2,mish,
                            linear3,mish,
                            linear4,mish,
                            linear5,mish,
                            linear6,mish,
                            linear7,mish,
                            linear8,mish,
                            linear9,mish,
                            linear10).to(device)

In [54]:
criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [55]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch
    print('epoch ','%04d' % (epoch+1), 'cost = ','{:.9f}'.format(avg_cost))
print('Learning finished..!')

epoch  0001 cost =  0.378584385
epoch  0002 cost =  0.135058209


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  if __name__ == '__main__':


epoch  0003 cost =  0.099514887
epoch  0004 cost =  0.067668095
epoch  0005 cost =  0.054085877
epoch  0006 cost =  0.046645936
epoch  0007 cost =  0.040235486
epoch  0008 cost =  0.038542077
epoch  0009 cost =  0.036857985
epoch  0010 cost =  0.035136871
epoch  0011 cost =  0.034404941
epoch  0012 cost =  0.033738106
epoch  0013 cost =  0.033007640
epoch  0014 cost =  0.032711979
epoch  0015 cost =  0.031921398
epoch  0016 cost =  0.031626508
epoch  0017 cost =  0.031127229
epoch  0018 cost =  0.031153087
epoch  0019 cost =  0.030485118
epoch  0020 cost =  0.029813962
epoch  0021 cost =  0.029740397
epoch  0022 cost =  0.029831186
epoch  0023 cost =  0.028933702
epoch  0024 cost =  0.028564271
epoch  0025 cost =  0.028284268
epoch  0026 cost =  0.027589817
epoch  0027 cost =  0.027565092
epoch  0028 cost =  0.028020460
epoch  0029 cost =  0.027725041
epoch  0030 cost =  0.027876448
epoch  0031 cost =  0.026376266
epoch  0032 cost =  0.025575697
epoch  0033 cost =  0.025249196
epoch  0

In [56]:
with torch.no_grad():
  x_test = test.iloc[:,1:7]
  x_test = np.array(x_test)

  x_test = torch.from_numpy(x_test).float().to(device)

  prediction = model(x_test)

In [57]:
correct_prediction = prediction.cpu().numpy().reshape(-1,1)

In [58]:
MAKE = pd.read_csv('Solar_TestData_2.csv', header = None, skiprows= 1) 

In [59]:
submit = pd.read_csv('Solar_SubmitForm_2.csv')
submit

,YYYY/MM/DD,Predict
0,0000-00-00,NaN
1,0000-00-00,NaN
2,0000-00-00,NaN
3,0000-00-00,NaN
4,0000-00-00,NaN
5,0000-00-00,NaN
6,0000-00-00,NaN
7,0000-00-00,NaN
8,0000-00-00,NaN
9,0000-00-00,NaN


In [60]:
for i in range(len(correct_prediction)):
  submit['Predict'][i] = correct_prediction[i].item()

submit['YYYY/MM/DD'] = MAKE[0]
submit

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,YYYY/MM/DD,Predict
0,2018-01-11,1.335570
1,2018-02-16,0.908475
2,2018-03-04,0.464941
3,2018-03-28,1.773736
4,2018-04-27,2.016506
5,2018-05-11,2.100078
6,2018-05-30,1.665926
7,2018-06-12,0.660516
8,2018-06-25,1.833445
9,2018-07-28,1.250774


In [61]:
submit.to_csv('result.csv', mode='w', index = False)

In [62]:
! kaggle competitions submit -c solarenergy-meteorologicalphenomenon2 -f result.csv -m "14010974 이기택"

100% 497/497 [00:05<00:00, 83.2B/s]
Successfully submitted to SejongAI.텀프로젝트.[기상현상에 따른 태양광발전량 예측]